In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from energyclustering.data.fluvius.data import read_data_pickle
from energyclustering.data.weather.data import read_weather_data
from energyclustering.sampling.preprocessing import preprocess_info_df_for_sampling_classification, yearly_profile_df_to_daily_df

# NOTE: now integrated into the code

### All the days

In [ ]:
info_df, data_df = read_data_pickle()
years = data_df.index.get_level_values('year').unique()
first_year, last_year = years.min(), years.max()
all_days = pd.date_range(f"1/1/{first_year}", f"31/12/{last_year}", freq = '1D')
day_df = pd.DataFrame(index = all_days)
day_df

### Information about days

In [ ]:
day_df = day_df.assign(
    day = lambda x: x.index.day, 
    month = lambda x: x.index.month, 
    year = lambda x: x.index.year, 
    day_of_week = lambda x: x.index.weekday,
    is_weekend = lambda x: x.index.day_of_week >=5,
    iso_day = lambda x: x.index.day_of_year,
    iso_week = lambda x: x.index.isocalendar().week,
)


import holidays
belgium_holidays = holidays.BE()
day_df['is_holiday'] = all_days.map(lambda date: date in belgium_holidays)
day_df

# Sample the days per week

From each week sample one weekday and one weekend day

In [ ]:
sampled_days = day_df[day_df.iso_week % 2 == 0].groupby(['year', 'iso_week', 'is_weekend']).apply(lambda x: x.sample(1)).droplevel([0,1,2]).sort_index()
sampled_days

In [ ]:
sampled_days_per_year = {key:value.apply(lambda x: x.replace(year = 2016)) for key,value in sampled_days.reset_index().groupby('year')['index']}
sampled_days_per_year;

## Subsample the days

In [ ]:
info_df = info_df.set_axis(info_df.index.to_flat_index().map(str), axis = 0)
data_df = data_df.set_axis(data_df.index.to_flat_index().map(str), axis = 0)

In [ ]:
def subsample_year(df): 
    year = int(df.name[-5:-1])
    days_to_sample = sampled_days_per_year[year]
    return df.loc[(df.name,days_to_sample), :]

In [ ]:
daily_df = yearly_profile_df_to_daily_df(data_df)
daily_df

In [ ]:
subsampled_df = daily_df.groupby(axis = 0, level = 0).apply(subsample_year).droplevel(0)
subsampled_df

In [ ]:
subsampled_df.shape[0]/daily_df.shape[0]

In [ ]:
2/7